# Вихідні дані `Court fair`

![title](https://i.ibb.co/KLPm3x1/Screenshot-1.png)

---

In [1]:
import os
import requests
import numpy as np
import pandas as pd
from time import sleep

In [2]:
def sample_data(
    chunksize=10**4, size=5, path="./raw_files/documents.csv" # https://court.gov.ua/opendata/
) -> pd.DataFrame:
    """
    Завантажує .csv файл шматочками й повертає лише певну його частину. 
    (Тому що не можу завантажити увесь файл одразу, для ознайомл. - ок)
    
    ---
    chunksize: по скільки рядків розбивати файл
         size: скільки частин файлу завантажити
    """

    samples = []
    for chunk in pd.read_csv(path, sep="\t", chunksize=chunksize):
        samples.append(chunk)

    high = len(samples)
    N = list(np.random.randint(low=0, high=high, size=size))
    print(
        f"actual size: ~{high * chunksize}, sample size: {size * chunksize}"
    )
    return pd.concat([samples[n] for n in N])


def download_files(dataframe, directory="./texts"):
    """
    Створює папку й завантажує в неї файли за посиланнями з dataframe
    """
    os.makedirs(directory, exist_ok=True)
    
    for url, name in zip(dataframe["doc_url"], dataframe["doc_id"]):
        r = requests.get(url, sleep(1))
        with open(f"{directory}/case_{name}.rtf", "wb") as f:
            f.write(r.content)

In [3]:
dataframes = sample_data(size=2)

actual size: ~5070000, sample size: 20000


In [6]:
test = dataframes.sample(5).copy()
test

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
1581349,80514589,2027,2,5,41080.0,643/3050/19,2019-03-18 00:00:00+02,2019-03-19 00:00:00+02,Довготько Т. М.,http://od.reyestr.court.gov.ua/files/40/fc4b8a...,1,2019-03-20 00:00:00+02
1581029,80507479,4855,5,4,5589.0,810/1595/16,2019-03-18 00:00:00+02,2019-03-19 00:00:00+02,Ключкович Василь Юрійович,http://od.reyestr.court.gov.ua/files/40/1119c2...,1,2019-03-20 00:00:00+02
3781744,82725897,520,5,1,40376.0,264/1159/19,2019-07-01 00:00:00+03,2019-07-02 00:00:00+03,Іванченко А. М.,http://od.reyestr.court.gov.ua/files/41/bd95e4...,1,2019-07-03 00:00:00+03
1588254,80517483,2315,5,1,40376.0,755/13171/18,2019-03-15 00:00:00+02,2019-03-19 00:00:00+02,Пасацька Л. А.,http://od.reyestr.court.gov.ua/files/40/cfe908...,1,2019-03-20 00:00:00+02
3780579,82732430,2027,5,1,40376.0,643/6248/19,2019-07-01 00:00:00+03,2019-07-02 00:00:00+03,Афанасьєв В. О.,http://od.reyestr.court.gov.ua/files/41/fb52e2...,1,2019-07-03 00:00:00+03


In [7]:
download_files(test)

### TO DO:
1. перетворити `.rtf` в `.txt`
2. відсортувати рішення за ключовими словами